# Tensorflow Basics

Notebook for taking notes and trialing out code for the Tensorflow 2.0 Basics tutorial (https://www.tensorflow.org/tutorials/quickstart/beginner).

In [ ]:
import tensorflow as tf

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10)
])

In [ ]:
# Don't bake in softmax to last layer activation as this may not be numerically stable
predictions = model(x_train[:1]).numpy()
tf.nn.softmax(predictions).numpy()

In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# Untrained model on one sample is close to random guess loss
print("Random guess loss: ", -tf.math.log(1/10).numpy()) # 10 categories
print("Our randomly initialised NN: ", loss_fn(y_train[:1], predictions).numpy())

In [ ]:
model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, epochs=5)

In [ ]:
model.evaluate(x_test, y_test, verbose=2)

In [ ]:
probability_model = tf.keras.Sequential([
    model, tf.keras.layers.Softmax()
])

In [ ]:
probability_model(x_test[:5])

## More complex dataset

Use the CIFAR10 dataset to try a similar modelling technique using keras etc.

This dataset contains 60,000 32x32 colour images in 10 classes.

In [ ]:
# Load the dataset
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [ ]:
# Extremely low resolution images
from matplotlib import pyplot as plt

print(y_train[1])
plt.imshow(X_train[1])

In [ ]:
import numpy

# Find the maximum pixel saturation
max_sat = numpy.amax(X_train)

# Normalise by the maximum saturation
training_data, test_data = tf.data.Dataset.from_tensor_slices(((X_train / max_sat), y_train)), tf.data.Dataset.from_tensor_slices(((X_test / max_sat), y_test))

In [ ]:
example = training_data.take(1)

In [ ]:
# Build a simple model using the Keras API
# Each image has 32 x 32 x 3 = 3072 values and last layer (producing the logits)
# requires 10 (for 10 categories).

model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(32, 32, 3)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10)
])

In [ ]:
# Define the loss function and compile the model with the loss function etc.

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss=loss_fn, metrics=["accuracy"])

In [ ]:
# Batch the dataset

batched_dataset = training_data.batch(500)

In [ ]:
# Fitting the model gives poor performance

model.fit(batched_dataset, batch_size=500, epochs=25)